# Introduction

**Note:** This information has been adapted from the ACS Data Handbook v9.0, [Chapter 3.5.2](http://www.stsci.edu/hst/acs/documents/handbooks/currentDHB/acs_Ch36.html#129340) Example 5.

**Warning:** Before running any code, please be sure you have installed and updated [AstroConda](https://astroconda.readthedocs.io/en/latest/) to the latest version.

To download this page as a Python 3 Jupyter Notebook, please visit the [acs-notebook GitHub page](https://github.com/spacetelescope/acs-notebook).

With the installation of the ASIC to replace the CCD electronics box during Servicing Mission 4 (SM4; May 2009), the ACS/WFC detector has low-level, horizontal striping present in the bias. This striping is removed as part of the `calacs` calibration pipeline for full-frame images, however de-striping is not performed automatically for subarray data. The pixel-based CTE correction cannot be performed on data with bias striping present, therefore ACS/WFC subarray data products retrieved from [MAST](https://archive.stsci.edu) will not be CTE-corrected. However, some subarray data can be corrected for CTE, which we will illustrate below.

Beginning in Cycle 24 (November 2016), the ACS Team implemented new flight software that changed the way CCD subarray images are read out (see ACS ISR [2017-03](http://www.stsci.edu/hst/acs/documents/isrs/isr1703.pdf)). This change makes it possible to apply the pixel-based CTE correction to subarray images all subarray data taken after this date. 

Subarray data can be calibrated, de-striped, and CTE-corrected using the `acs_destripe_plus` code within the [ACSTOOLS](https://github.com/spacetelescope/acstools) Python package. This tool can be used to calibrate ACS/WFC subarray data in the following cases:

1. 2K subarrays taken after SM4
2. All other subarrays taken after the start of Cycle 24

Note that the `acs_destripe_plus` code acts as a wrapper around the `calacs` code, performing all of the calibration steps that would normally be performed (i.e., bias/dark subtraction, flat-fielding, etc.). As such, this tool is used on `*_raw.fits` files.

# Example

Our example data come from an observation of the globular cluster 47 Tuc, which is often used for ACS calibration. We will use the `astroquery` tool to download the dataset automatically. The association name is JD5702011, and it is comprised of two images in a CR-SPLIT pattern using the WFC1A-1K 1024x1024 subarray. The example will only use the first image with the rootname JD5702JMQ.

### Download the Data

Note that the following code is simply for downloading the `*_raw.fits` file and moving it to the current working directory. If the file is already located in the current working directory, you do not need to repeat this step.

In [ ]:
from astroquery.mast import Observations
import shutil

Observations.download_products('2007656106', mrp_only=False, productSubGroupDescription=['RAW', 'UNCAL'], 
                              extension='fits', cache=True)

shutil.move('mastDownload/HST/JD5702JMQ/jd5702jmq_raw.fits', '.')

shutil.rmtree('mastDownload')

### Update Header Keywords

Next we will need to turn on the CTE correction switch in the primary header. We will also need to supply the names of the `PCTETAB` and `DRKCFILE` files, which are the CTE correction parameters and the CTE-corrected dark reference files, respectively. This information can be retrieved using the Python CRDS interface. Note that we have prefaced the names of the reference files with '`jref$`', which is the environment variable pointing to the directory containing ACS calibration data. You must set the JREF environment variable prior to proceding. This example assumes that you have placed the appropriate calibration reference files in your JREF directory. Visit the [CRDS](http://www.stsci.edu/hst/observatory/crds/) website to download calibration data.

In [ ]:
from astropy.io import fits
from crds import getrecommendations
from crds import client

with fits.open('jd5702jmq_raw.fits') as hdu:
    date = hdu[0].header['DATE-OBS']
    time = hdu[0].header['TIME-OBS']
    amps = hdu[0].header['CCDAMP']
    gain = hdu[0].header['CCDGAIN']

client.set_crds_server('https://hst-crds.stsci.edu')
params = {'INSTRUME': 'ACS', 'DETECTOR': 'WFC', 'DATE-OBS': date, 'TIME-OBS': time, 
          'CCDAMP': amps, 'CCDGAIN': gain}
reffiles = getrecommendations(parameters=params, reftypes=['PCTETAB', 'DRKCFILE'], 
                              observatory='hst')

with fits.open('jd5702jmq_raw.fits', mode='update') as hdu:
    hdu[0].header['PCTECORR'] = 'PERFORM'
    hdu[0].header['PCTETAB'] = 'jref$' + reffiles['pctetab']
    hdu[0].header['DRKCFILE'] = 'jref$' + reffiles['drkcfile']

### Run acs_destripe_plus

Next we will run the `acs_destripe_plus` code on our image. This will execute all of the calibration steps that are set to "PERFORM" in the primary header of the FITS file. Note that with the CTE correction turned on, this can take some time. On an 8 core machine, the CTE correction will take approximately 3 minutes to complete, and will use all of the cores. The `acs_destripe_plus` code will produce two new files: `jd5702jmq_flt.fits` and `jd5702jmq_flc.fits`. The FLT file is the usual output product from `calacs`, while the FLC file is the calibrated FLT image with the CTE correction applied.

In [ ]:
from acstools import acs_destripe_plus

acs_destripe_plus.destripe_plus('jd5702jmq_raw.fits',cte_correct=True)

### Correct the WCS

The subarray products produced by this process do not have the proper WCS information in the header. The WCS information is normally updated by the pipeline via an additional call to AstroDrizzle. Here, we can manually update the WCS of our FLC product using `stwcs.updatewcs`. 

In [ ]:
from stwcs import updatewcs

updatewcs.updatewcs('jd5702jmq_flc.fits')